<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/NER_name_gender_ethnicity_redact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q
!pip install langchain -q
!wget -q -O - ipv4.icanhazip.com
!pip install racebert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.5 MB/s eta 0:00:00
35.201.210.47


In [2]:
# Bring in deps
import os
import streamlit as st
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper

#for the model, use Hugging face pipeline
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

Design a MVP for masking Name and synthesizing Name according to his/her gender and enthinicity.
    #1. Streamlit for UI (same as Presidio)
    #2. Langchain for combining models, and other packages

Streamlit setup

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.122.82.32


In [ ]:
# App framework
# save it as app.py
import os
import streamlit as st
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper

#for the model, use Hugging face pipeline
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

tokenizer_ner = AutoTokenizer.from_pretrained("bigcode/starpii", model_max_length=512)
model_ner = AutoModelForTokenClassification.from_pretrained("bigcode/starpii")
tokenizer_ethnicity = AutoTokenizer.from_pretrained("pparasurama/raceBERT-ethnicity", model_max_length=512)
model_ethnicity = AutoModelForTokenClassification.from_pretrained("pparasurama/raceBERT-ethnicity")

from racebert import RaceBERT
model_race = RaceBERT()

def ner_name(text, model, tokenizer):
    # tokenizer = AutoTokenizer.from_pretrained("bigcode/starpii", model_max_length=512)
    # model = AutoModelForTokenClassification.from_pretrained("bigcode/starpii")
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)
    entities = ner_pipeline(text)
    return entities

def ner_entity_to_fullname(entities):

    full_name = ''
    for entity in entities:
        if entity['entity'] in ['B-NAME', 'I-NAME']:
            if entity['word'].startswith('Ġ'):
                full_name += ' '
                full_name += entity['word'].replace('Ġ', '')
            else:
                full_name += entity['word']
    return full_name.strip()

def fullname_to_gender(fullname):
    import requests, json
    first_name = fullname.split()[0]
    content = requests.get(f"https://api.genderize.io/?name={first_name}").text
    gender = json.loads(content)['gender']
    return gender

def fullname_to_enthnicity(fullname, model, tokenizer):
    # tokenizer = AutoTokenizer.from_pretrained("pparasurama/raceBERT-ethnicity", max_length=512)
    # model = AutoModelForTokenClassification.from_pretrained("pparasurama/raceBERT-ethnicity")
    ethnicity_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)
    ethnicity = ethnicity_pipeline(fullname)
    return ethnicity[0]['label'].split(',')[0]

def fullname_to_race(fullname, model):
    # from racebert import RaceBERT
    # model = RaceBERT()
    race = model.predict_race(fullname)
    race_map = {'nh_white':'white',
               'hispanic':'hispanic',
               'nh_black':'black',
               'api': 'asian',
               'aian': 'american indian'
        }
    race = race_map[race[0]['label']]
    return race

def redact_name(text):
    # my name is Ryan Chen to my name is asian male
    entities = ner_name(text, model_ner, tokenizer_ner)
    if not entities:
        return text
    fullname = ner_entity_to_fullname(entities)
    gender = fullname_to_gender(fullname)
    race = fullname_to_race(fullname, model_race)

    #pos of name, +1 for special token 'Ġ' of capital letter
    pos_start = entities[0]['start'] + 1
    pos_end = entities[-1]['end']
    redacted_text = text[:pos_start-1] + ' ' + race + ' ' + gender + ' (redacted)' + text[pos_end:]
    return redacted_text

# main body
# takes more time to download models in the first run.
st.title('Juniper: Name Entity Redacting')
prompt = st.text_input('Plug in your prompt here')

if prompt:
    redacted_name = redact_name(prompt)
    st.write(redacted_name)

In [ ]:
# run the app, copy url and paste the ip we got to get access to the streamlit webpage.
!streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.122.82.32:8501

npx: installed 22 in 4.142s
your url is: https://evil-pears-lead.loca.lt
/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py:548: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import OpenAI`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/utilities/__init__.py:24: LangChainDeprecationWarning: Importing this utility from langchain is deprecated. Importing it from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langc

1. Call "bigcode/starpii" BERT for NER task
2. Call genderiz.io api for gender information
3. Call raceBERT model or raceBERT-ethnicity for ethnicity information
4. Call firstM, firstF python file for name generating if needed

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#Get the NER BERT model from Hugging face
#use default tokenizer and
ner_pipeline = pipeline("ner", model="bigcode/starpii")
text = "My name is Ryan Chen"
entities = ner_pipeline(text)
print(entities)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/494M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/975 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-NAME', 'score': 0.999767, 'index': 3, 'word': 'ĠR', 'start': 10, 'end': 12}, {'entity': 'I-NAME', 'score': 0.9997906, 'index': 4, 'word': 'yan', 'start': 12, 'end': 15}, {'entity': 'I-NAME', 'score': 0.99888223, 'index': 5, 'word': 'ĠChen', 'start': 15, 'end': 20}]


In [23]:
# name-gender from genderize.io using its API
import requests, json

name = 'Ryan'
content = requests.get(f"https://api.genderize.io/?name={name}").text
gender = json.loads(content)['gender']

print(gender)

None


In [ ]:
#Name generator
#upload male's and female's name and firstF.py, firstM.py
#female's name .txt files are in f_names folder
from firstF import FirstNameFemale
from firstM import FirstNameMale

FirstNameMale('american')

'Bellamy'

In [36]:
from racebert import RaceBERT

model = RaceBERT()

# To predict race
model.predict_race("Michael Jordan")


# Race	Label
# Non-hispanic White	nh_white
# Hispanic	hispanic
# Non-hispanic Black	nh_black
# Asian & Pacific Islander	api
# American Indian & Alaskan Native	aian


[{'label': 'nh_white', 'score': 0.7121939063072205}]

In [5]:
#Get the ethnicity BERT model from Hugging face
ethnicity_pipeline = pipeline("text-classification", model="pparasurama/raceBERT-ethnicity")
name = "michael jordan"
result = ethnicity_pipeline(name)
print(result)

[{'label': 'GreaterEuropean,British', 'score': 0.2740285396575928}]


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("pparasurama/raceBERT-ethnicity")

# Attempt to print out the label names
print(model.config.id2label)

{0: 'GreaterEuropean,British', 1: 'GreaterEuropean,WestEuropean,French', 2: 'GreaterEuropean,WestEuropean,Italian', 3: 'GreaterEuropean,WestEuropean,Hispanic', 4: 'GreaterEuropean,Jewish', 5: 'GreaterEuropean,EastEuropean', 6: 'Asian,IndianSubContinent', 7: 'Asian,GreaterEastAsian,Japanese', 8: 'GreaterAfrican,Muslim', 9: 'Asian,GreaterEastAsian,EastAsian', 10: 'GreaterEuropean,WestEuropean,Nordic', 11: 'GreaterEuropean,WestEuropean,Germanic', 12: 'GreaterAfrican,Africans'}


In [41]:
tokenizer_ner = AutoTokenizer.from_pretrained("bigcode/starpii", model_max_length=512)
model_ner = AutoModelForTokenClassification.from_pretrained("bigcode/starpii")
tokenizer_ethnicity = AutoTokenizer.from_pretrained("pparasurama/raceBERT-ethnicity", model_max_length=512)
model_ethnicity = AutoModelForTokenClassification.from_pretrained("pparasurama/raceBERT-ethnicity")

from racebert import RaceBERT
model_race = RaceBERT()

def ner_name(text, model, tokenizer):
    # tokenizer = AutoTokenizer.from_pretrained("bigcode/starpii", model_max_length=512)
    # model = AutoModelForTokenClassification.from_pretrained("bigcode/starpii")
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)
    entities = ner_pipeline(text)
    return entities

def ner_entity_to_fullname(entities):

    full_name = ''
    for entity in entities:
        if entity['entity'] in ['B-NAME', 'I-NAME']:
            if entity['word'].startswith('Ġ'):
                full_name += ' '
                full_name += entity['word'].replace('Ġ', '')
            else:
                full_name += entity['word']
    return full_name.strip()

def fullname_to_gender(fullname):
    import requests, json
    if len(fullname.split()) == 1:
        first_name = fullname
    else:
        first_name = fullname.split()[0]
    content = requests.get(f"https://api.genderize.io/?name={first_name}").text
    gender = json.loads(content)['gender']
    return gender

def fullname_to_enthnicity(fullname, model, tokenizer):
    # tokenizer = AutoTokenizer.from_pretrained("pparasurama/raceBERT-ethnicity", max_length=512)
    # model = AutoModelForTokenClassification.from_pretrained("pparasurama/raceBERT-ethnicity")
    ethnicity_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)
    ethnicity = ethnicity_pipeline(fullname)
    return ethnicity[0]['label'], ethnicity[0]['score']

def fullname_to_race(fullname, model):
    # from racebert import RaceBERT
    # model = RaceBERT()
    race = model.predict_race(fullname)
    race_map = {'nh_white':'white',
               'hispanic':'hispanic',
               'nh_black':'black',
               'api': 'asian',
               'aian': 'american indian'
        }
    race, score = race_map[race[0]['label']], race[0]['score']
    return race, score

def redact_name(text):
    # my name is Ryan Chen to my name is asian male
    entities = ner_name(text, model_ner, tokenizer_ner)
    if not entities:
        return text
    fullname = ner_entity_to_fullname(entities)
    gender = fullname_to_gender(fullname)
    race, score = fullname_to_race(fullname, model_race)

    #pos of name, +1 for special token 'Ġ' of capital letter
    pos_start = entities[0]['start'] + 1
    pos_end = entities[-1]['end']
    redacted_text = text[:pos_start-1] + ' ' + race + " " + gender + ' (redacted)' + text[pos_end:] + f'\nDisclaimer: The accuracy of race is {round(score, 2) * 100}% and of gender is 92.5%. This race and gender prediction tool is designed for supplementary use in medical diagnosis contexts and should be used with caution, understanding its limitations and the importance of not replacing professional judgment and comprehensive patient assessments.'
    return redacted_text

In [ ]:
text = "The legendary basketball player who dominated the sport during the 1990s, Michael Jordan, is widely admired."
entities = ner_name(text, model_ner, tokenizer_ner)
print(entities)
fullname = ner_entity_to_fullname(entities)
print(fullname)
gender = fullname_to_gender(fullname)
print(gender)
race, score = fullname_to_race(fullname, model_race)
print(race, str(round(score, 2)))


import time
start_time = time.time()

redacted_name = redact_name(text)
print(redacted_name)

end_time = time.time()
print(f"The function took {end_time - start_time} seconds to run.")

[{'entity': 'B-NAME', 'score': 0.9998375, 'index': 21, 'word': 'ĠMichael', 'start': 73, 'end': 81}, {'entity': 'I-NAME', 'score': 0.99961495, 'index': 22, 'word': 'ĠJ', 'start': 81, 'end': 83}, {'entity': 'I-NAME', 'score': 0.99980193, 'index': 23, 'word': 'ordan', 'start': 83, 'end': 88}]
Michael Jordan
male
white 0.76
The legendary basketball player who dominated the sport during the 1990s, white male (redacted), is widely admired.
Disclaimer: The accuracy of race is 76.0% and of gender is 92.5%. This race and gender prediction tool is designed for supplementary use in medical diagnosis contexts and should be used with caution, understanding its limitations and the importance of not replacing professional judgment and comprehensive patient assessments.
The function took 1.0647428035736084 seconds to run.


In [ ]:
#In the latest SpaceX launch, Elon Musk announced that the Starship would be ready for a manned mission to Mars by 2024.